## Annotation Visualizations

### Imports & Initializations 

In [1]:
import json
import os

In [2]:
def tokenize(text):
    # Tokenize based on spaces while keeping track of character positions.
    tokens = []
    char_indices = []
    start = 0
    for word in text.split():
        end = start + len(word)
        tokens.append(word)
        char_indices.append((start, end))
        start = end + 1  # Assuming a single char for space
    return tokens, char_indices

def align_labels_with_tokens(char_indices, labels_range):
    # Initialize labels for each token with 'O' (no label)
    labels = ['O'] * len(char_indices)
    
    for start, end, label in labels_range:
        for i, (tok_start, tok_end) in enumerate(char_indices):
            if not (end < tok_start or start > tok_end):  # Overlap condition
                labels[i] = label
    return labels

def extract_dataset_labels(json_data):
    id_to_label = {0: 'O', 1: 'Organization', 2: 'O', 3: 'Person', 4: 'Person', 5: 'Location', 6: 'Organization', 7: 'O', 8: 'Location'}
    
    test_labels = {}
    labels = {}
    label_path = os.path.join( "data", "conll2003", "dataset", "test.json")

    with open(label_path, "r") as f:
        for line in f:
            # Each line is a separate JSON object
            json_line = json.loads(line)
            text = ""
            for word in json_line["tokens"]:
                text += word + " "
            test_labels[text.strip()] = json_line["tags"]

    for task in json_data["tasks"]:
        text = task["data"]["text"].strip()
        if text in list(test_labels.keys()):
            label = []
            for _label in test_labels[text]:
                label.append(id_to_label[_label])
            labels[text] = label
    return labels

def extract_model_labels(json_data):
    labels = {}
    for task in json_data["tasks"]:
        text = task["data"]["text"]
        tokens, char_indices = tokenize(text)
        pred_ranges = [(r["value"]["start"], r["value"]["end"], r["value"]["labels"][0]) for p in task["predictions"] for r in p["result"]]
        pred_labels = align_labels_with_tokens(char_indices, pred_ranges)
        labels[text] = pred_labels
    return labels

def extract_annotations_labels(json_data):
    labels = {}
    for task in json_data["tasks"]:
        text = task["data"]["text"]
        tokens, char_indices = tokenize(text)
        anno_ranges = [(r["value"]["start"], r["value"]["end"], r["value"]["labels"][0]) for a in task["annotations"] for r in a["result"]]
        anno_labels = align_labels_with_tokens(char_indices, anno_ranges)
        labels[text] = anno_labels
    return labels

def extract_inital_predictions_labels(json_data):
    labels = {}
    for task in json_data["tasks"]:
        text = task["data"]["text"]
        tokens, char_indices = tokenize(text)
        pred_ranges = [(r["value"]["start"], r["value"]["end"], r["value"]["labels"][0]) for p in task["predictions"] for r in p["result"]]
        pred_labels = align_labels_with_tokens(char_indices, pred_ranges)
        labels[text] = pred_labels
    return labels

def calculate_accuracy(prediction_labels, true_labels, per_sentence=False):
    if per_sentence:
        for text in prediction_labels:
            correct = 0
            for pred, true in zip(prediction_labels[text], true_labels[text]):
                if pred == true:
                    correct += 1
            print(f"Accuracy: {correct/len(prediction_labels[text])}")
    else:
        correct = 0
        total = 0
        for text in prediction_labels:
            for pred, true in zip(prediction_labels[text], true_labels[text]):
                if pred == true:
                    correct += 1
                total += 1
        print(f"Accuracy: {correct/total}")

In [3]:
trial_type = "solo"
annotation_path = os.path.join("data", f"{trial_type}", "annotations")
annotations = {}
inital_model_predictions = {}

for file in os.listdir(annotation_path):
    if file.endswith("_annotations.json"):
        with open(os.path.join(annotation_path, file), "r") as f:
            data = json.load(f)
            participant_id = str(file.split("_")[1])
            annotations[participant_id] = data
    elif file.startswith("model_predictions"):
        with open(os.path.join(annotation_path, file), "r") as f:
            data = json.load(f)
            inital_model_predictions = data

In [4]:
inital_predictions_labels = extract_inital_predictions_labels(inital_model_predictions)
print(inital_predictions_labels)

{'SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .': ['Person', 'O', 'Person', 'O', 'O', 'O', 'O', 'Person', 'O', 'O', 'O', 'O'], 'Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .': ['Organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Organization', 'O', 'O', 'Organization', 'O', 'O', 'O', 'O', 'O', 'O'], 'But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers Uzbekistan .': ['O', 'Organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .': ['Organization', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

### Accuracies

In [5]:
dataset_labels = extract_dataset_labels(inital_model_predictions)
inital_model_labels = extract_model_labels(inital_model_predictions)

print("Initial model accuracy with respect to dataset:")
calculate_accuracy(inital_model_labels, dataset_labels, per_sentence=False)

print()

for participant in annotations.keys():
    print(f"Participant {participant}")
    updated_model_labels = extract_model_labels(annotations[participant])
    annotation_labels = extract_annotations_labels(annotations[participant])
    
    print("Annotation accuracy with respect to dataset:")
    calculate_accuracy(annotation_labels, dataset_labels, per_sentence=False)

    print("Updated model accuracy with respect to dataset:")
    calculate_accuracy(updated_model_labels, dataset_labels, per_sentence=False)

    print("Updated model accuracy with respect to annotation:")
    calculate_accuracy(updated_model_labels, annotation_labels, per_sentence=False)

    print("Initial model accuracy with respect to annotation:")
    calculate_accuracy(inital_predictions_labels, annotation_labels, per_sentence=False)

    print()


Initial model accuracy with respect to dataset:
Accuracy: 0.9009433962264151

Participant 001
Annotation accuracy with respect to dataset:
Accuracy: 0.9528301886792453
Updated model accuracy with respect to dataset:
Accuracy: 0.9009433962264151
Updated model accuracy with respect to annotation:
Accuracy: 0.8773584905660378
Initial model accuracy with respect to annotation:
Accuracy: 0.8773584905660378

Participant 002
Annotation accuracy with respect to dataset:
Accuracy: 0.9457547169811321
Updated model accuracy with respect to dataset:
Accuracy: 0.9009433962264151
Updated model accuracy with respect to annotation:
Accuracy: 0.8679245283018868
Initial model accuracy with respect to annotation:
Accuracy: 0.8679245283018868

Participant 004
Annotation accuracy with respect to dataset:
Accuracy: 0.9221698113207547
Updated model accuracy with respect to dataset:
Accuracy: 0.9009433962264151
Updated model accuracy with respect to annotation:
Accuracy: 0.8867924528301887
Initial model accur